In [2]:
from matplotlib import pyplot as plt
from numpy import linspace
import numpy as np
import scipy
import scipy.optimize
from cmath import *

# Parameters calculation

In [3]:
#Defenition of all matrices/params
pi = 3.14159265359
hbar = 1.054e-34
fi_0 = 2.07e-15/2/np.pi

#ALSO - mb add non-Jos inductance?

C_J = 0.48e-12#0.8e-12
#I_J =  2.7e-6
L_J = 0.12e-9#0.03e-9#fi_0 / I_J
 
N = 80#300#80
    
C_out = 22e-15
C_0 = .5e-15


print L_J*1e9, 'nH'
print (C_J*L_J)**(-0.5) /6.28e9, 'GHz'
################################

C = []
L = []# ^-1

for i in range(N+1):
    C.append([])
    L.append([])
    for j in range(N+1):
        if ((i==0)&(j==0)):
            #L[i].append(1./L_J) #lambda/2 
            #C[i].append(C_J)    #lambda/2
            L[i].append(2./L_J)
            C[i].append(2*C_J + C_0)
            continue
        
        if ((i==N)&(j==N)):
            L[i].append(1./L_J) #correctr
            C[i].append(C_J + C_out)
            #L[i].append(2./L_J)
            continue
            
        if (i==j):
            L[i].append(2./L_J)
            C[i].append(2*C_J + C_0)
            continue
            
        if ((j==i+1)or(j==i-1)):
            L[i].append(-1./L_J)
            C[i].append(-C_J)
            continue
        L[i].append(0.)
        C[i].append(0.)

L = np.array(L)
C_inv = scipy.linalg.inv(np.array(C))

c_buf = np.diagonal(C)
l_buf = np.diagonal(L)

w = []
for k in range(N+1):
    w.append( (c_buf[k] * l_buf[k])**0.5 )

0.12 nH
20.9811415882 GHz


# Going from discrete to continious (w/o C_out)

In [266]:
n = linspace(0, N-1, N)
fr = []

def ka(n):
    return( (n + .5) * pi / N )


#def fun(k):
#    return( (2*L_J / pi**2 * (0.5*C_0*N**2 /k**2 + 0.5* pi**2*C_J ) )**-0.5)
def fun(k):
    return ( (L_J * C_J + C_0 * L_J / ka(k)**2)**-0.5 )
              
plt.plot(n+1, fun(n)/6.28e9, 'ro')
fr = fun(n)
plt.xlim(0,10)
plt.show()

In [ ]:
#/TODO
#MB include C_out at the end as Eichler did????

# Luca's calculations

In [4]:
C_inv_sqrt= np.mat(scipy.linalg.sqrtm(C_inv))
L_m = np.mat(L)

A=C_inv_sqrt*L*C_inv_sqrt

(freqs,psi) = scipy.linalg.eigh(A,overwrite_a=True)

mode = np.sqrt(freqs)
nums = linspace(1,N+1, N + 1)

plt.plot(nums,(mode)/2/np.pi/1e9,'ro')
plt.xlim(0,10)
plt.show()

In [5]:
#Quest - are Luca's calculations correct????
#/TODO - check them

modeNumber = 0

def Phi(k):#Correct??????
    return(C_inv_sqrt*psi *np.sqrt(hbar)/mode[k]**0.5)


psi_norm = C_inv_sqrt* psi 

psi_arr = np.array(psi.T)


phase_norm_luca = np.array((psi_norm * ((2*np.pi)/fi_0)*np.sqrt(hbar)/2/np.array(mode)**0.5).T) #Luca's numbers

phase_norm_quanta = np.array((psi_norm * ((2*np.pi)*fi_0)**-1 *np.sqrt(hbar)/2/np.array(mode)**0.5).T)

plt.plot(nums,(phase_norm_quanta[modeNumber]),'go', label = 'F/F_0')
plt.plot(nums,(phase_norm_luca[modeNumber]),'ro', label = 'Luca')

plt.legend(loc = 'best')
#plt.plot(nums,psi_arr[0],'ro')
plt.show()
#print freqs[]**0.5/6.28e9

## Nonlinearity calculation

In [6]:
alpha =  - 0.5* hbar/(24. * L_J * fi_0**2)

bla = np.array(C_inv_sqrt)

def eta_inside(x,j):#Care about psi_arr[j][y]
    ret = 0.
    for y in range(N + 1):
        ret = ret + C_J**0.5 * (bla[x+1][y] - bla[x][y])*np.array(psi_arr)[j][y]
    return(ret)

def eta(j,k,l,m): #Fine, same as Luca had
    ret = 0
    for x in range(N):
        ret = ret + eta_inside(x,j) * eta_inside(x,k) * eta_inside(x,l) * eta_inside(x,m)
    return(ret)

def beta(j,k,l,m):
    return alpha *eta(j,k,l,m) / (mode[j])**0.5 / (mode[k])**0.5 / (mode[l])**0.5/ (mode[m])**0.5 / C_J**2


In [7]:
beta(0,0,0,0)

-79566.36705747954

In [12]:
buff = []
eta_nums = linspace(0,10,11)
for k in range(11):
    buff.append(eta(k,k,k,k))
plt.plot(eta_nums, buff)
plt.show()

In [312]:
# K_p = 12 * beta(1,1,1,1)
# K_ps = 24* beta(1,1,2,2)
# K_pi = 24 * beta(1,1,0,0)
# K_psi = 24 * beta(1,1,2,0)

pump_mode = 3
signal_mode = 
idler_mode = 2

K_p = 12 * beta(pump_mode,pump_mode,pump_mode,pump_mode)
K_ps = 24* beta(pump_mode,pump_mode,signal_mode,signal_mode)
K_pi = 24 * beta(pump_mode,pump_mode,idler_mode,idler_mode)
K_psi = 24 * beta(pump_mode,pump_mode,signal_mode,idler_mode)

In [375]:
print 'K/1e6', K_p/1e3, K_ps/1e3, K_pi/1e3, K_psi/1e3

K/1e6 -0.15 1.2810624129 0.881064411029 0.423410477792


In [374]:
#Between-mode pumping
pump_mode = 2
signal_mode = 3
idler_mode = 2

s = signal_mode
i = idler_mode
K_psi = 24.* beta(s,s,i,i) + 24.* beta(s,s,s,i) + 12.* beta(s,i,i,i)
K_ps = 24.* beta(s,s,s,s) + 48.*beta(s,s,s,i) + 24.*beta(s,s,i,i)
K_pi = 24.* beta(i,i,i,i) + 48.*beta(s,i,i,i) + 24.*beta(s,s,i,i)

# Amplification calc

In [376]:
#Constants

pump_mode = 3
signal_mode = 4
idler_mode = 3

w_s0 = mode[signal_mode]
w_i0 = mode[idler_mode]
w_p0 = mode[pump_mode]
w_p_real = (w_s0 + w_i0)/2.


kappa_i = w_i0/20.
kappa_p = kappa_i * (w_i0/w_p0)**0.5
kappa_s = kappa_i * (w_i0/w_s0)**0.5

#K_p =  -0.15e3
#K_ps = -0.25e3
#K_pi = -0.145e3
#K_psi = -0.3e3

#-0.416502243384 -0.859543780353 -0.192102150416 0.216179482024 #- for N=300& Luca's params, K = number*1e6

print kappa_i/2e6/np.pi, kappa_p/2e6/np.pi, kappa_s/2e6/np.pi
print K_p/1e6, K_ps/1e6, K_pi/1e6, K_psi/1e6
print w_i0/2e9/np.pi, w_p0/2e9/np.pi, w_s0/2e9/np.pi,w_p_real/w_p0

503.711437762 503.711437762 451.054461748
-0.00015 0.0012810624129 0.000881064411029 0.000423410477792
10.0742287552 10.0742287552 12.5636973734 1.12355628796


In [364]:
print (w_p_real - w_i0)/kappa_s

2.75960979137


Pump duffing

In [391]:
##################################################
#Created to check if power is ok                ##
##################################################
w_p_real = (w_s0 + w_i0)/2

#Delete That later

w_p0 = w_s0
kappa_p = kappa_s
K_p = -K_ps



##################


freq_min = w_p0 - 10 * kappa_p
freq_max = w_p0 + 10 * kappa_p
freq_num = 1e3


def getPumpAlpha(alpha_in, w_p):
    def f(x,w_p):
        return( (  (w_p - w_p0)**2 + 0.25*kappa_p**2 )*x - 2*(w_p - w_p0)*K_p*x**2 + K_p**2 * x**3 - kappa_p * alpha_in**2 )
    
    a = (scipy.optimize.bisect(lambda x: f(x, w_p), 0,1e9, xtol=1e-17))**0.5
    
    return(a)

###TODO
#check all signs where kappa involved


#Rough search
#for alpha_log in linspace(-30, 30, 10):
 #   alpha_in = 10**(0.1*alpha_log)
    
#Smooth tuning
for alpha_in in linspace(.2e8, 1e8, 10):

    def f(x,w_p):
        return( ( (w_p - w_p0)**2 + 0.25*kappa_p**2 )*x - 2*(w_p - w_p0)*K_p*x**2 + K_p**2 * x**3 - kappa_p * alpha_in**2 )

    n_array = []
    for f_p in linspace(freq_min, freq_max, freq_num):
        n_array.append((scipy.optimize.bisect(lambda x: f(x, f_p), 0,1e9, xtol=1e-17))**0.5)

    bufer = np.array(n_array)
    f_p = linspace(freq_min - w_p0, freq_max - w_p0, freq_num)

    plt.plot( (f_p)/kappa_p, bufer, label = str(alpha_in) )
    

#plt.xlim(-2,0)
plt.axvline((w_p_real - w_p0)/kappa_p)
plt.xlabel('(f_p - f_p0)/kappa_p')
plt.ylabel('|alpha|')
plt.title('')
#plt.legend()
plt.show()

In [ ]:
getPumpAlpha(5e5, w_p0 - 0.7*kappa_p)

In [381]:
def W(w, w_0, kappa ,K, alpha):
    return( 1j * (w - w_0) + 0*1j - 0.5*kappa - 0.5j * K * alpha**2 )

def Gs(ws, w_p,alpha):
    w_i = 2*w_p - w_s
    W_i = W(w_i, w_i0, kappa_i, K_pi, alpha).conjugate()
    W_s = W(w_s, w_s0, kappa_s, K_ps, alpha)
    
    g = -1. - kappa_s * W_i / ( 0*1j + W_s * W_i - 0.25 * K_psi**2 * alpha**4 )
    return(20*np.log10(abs(g)))
    #return g
    
def Gi(ws, w_p,alpha):
    w_i = 2*w_p - w_s
    W_i = W(w_i, w_i0, kappa_i, K_pi, alpha).conjugate()
    W_s = W(w_s, w_s0, kappa_s, K_ps, alpha)

    i = 0.5j * (kappa_s*kappa_i)**0.5 * K_psi*alpha**2/ (0*1j + W_s * W_i - 0.25 * K_psi**2 * alpha**4 )
    return(20*np.log10(abs( i )))
    #return i

In [ ]:
#TODO
#Write down code which will plot both Gain(pump parameters) and duffung curves of pump simultaneously!!!!! 

In [386]:
#Wild guessies

#w_p = w_p0 - 0.7*kappa_p
w_s = linspace(w_s0 - 15*kappa_s, w_s0 + 15*kappa_s, 1e4)

#getPumpAlpha(5e5, w_p0 - 0.7*kappa_p)
alpha = 3e3
for w_p in linspace(w_p_real - 3*kappa_p, w_p_real + 3* kappa_p, 100):
    #for alpha_in in linspace(1e4,4e5,10):
        #alpha = 10**alpha_pow
        #alpha = getPumpAlpha(alpha_in, w_p)
        #print alpha
    plt.plot((w_s - w_s0)/kappa_s,Gs(w_s, w_p, alpha))
        #plt.plot((w_s - w_s0)/kappa_s, abs(Gs(w_s, w_p, alpha))**2 - abs(Gi(w_s, w_p, alpha))**2)
    #plt.ylim(0.,2.)
plt.show()

In [332]:
#Real alpha from pump
w_p = w_p0 - 0.7*kappa_p
w_s = linspace(w_s0 - 5*kappa_s, w_s0 + 5*kappa_s, 1e4)

#getPumpAlpha(5e5, w_p0 - 0.7*kappa_p)

for w_p in linspace(w_p0 - 3*kappa_p, w_p0 + 1* kappa_p, 30):
    for alpha_in in linspace(1e8,2e8,30):
        #alpha = 10**alpha_pow
        alpha = getPumpAlpha(alpha_in, w_p)
        #print alpha
        plt.plot((w_s - w_s0)/kappa_s,Gs(w_s, w_p, alpha))
        #plt.plot((w_s - w_s0)/kappa_s, abs(Gs(w_s, w_p, alpha))**2 - abs(Gi(w_s, w_p, alpha))**2)
    #plt.ylim(0.,2.)
plt.show()

In [ ]:
#Check
def PumpShifter(alpha):
    return((w_s0 + w_i0 + 0.5 * alpha**2 * (K_ps + K_pi))/2. )

def PumpFreqShift(alpha):
    return(w_p0 + K_p*alpha**2)

alpha = linspace(1e0, 1e2, 5e1)
plt.plot(alpha,(PumpShifter(alpha) - PumpFreqShift(0.)) /kappa_p, 'g', label = 'RealPump')
plt.plot(alpha,(PumpFreqShift(alpha) - PumpFreqShift(0.)) /kappa_p, 'r', label = 'Pump Resonant Freq')
plt.legend(loc = 3)
plt.xlabel('alpha')
plt.ylabel( '(Freq - Freeq_real0) / kappa_p')
plt.axhline(-1.)
plt.show()

In [ ]:
a = [[1,2],[3,4]]
b = [[2,2], [0,0]]
np.dot(a,b)

# Pumping between resonancies

In [379]:
alpha_in = 1e5

w = linspace(3* 6.28e9, 13* 6.28e9,1e5)
w_r1 = 10* 6.28e9
w_r2 = 12 * 6.28e9

kappa_1 = 150e6* 6.28
kappa_2 = 100e6* 6.28


def responce(w,w_r, kappa):
    return(1* kappa**0.5 / ( 0.5 * kappa + 1j*(-w + w_r) ))

plt.plot(w/6.28e9, 20* np.log10(abs(responce(w, w_r1, kappa_1))))
plt.plot(w/6.28e9, 20* np.log10(abs(responce(w, w_r2, kappa_2))))
plt.axvline((w_r1 + w_r2)/2/6.28e9)
plt.ylabel('|alpha/alpha_in|')
plt.show()